### Ridership Clean Up

In [1]:
import pandas as pd
import numpy as np

In [2]:
ridership_2014 = pd.read_csv('csv_originals/2014_platform_usage.csv')
ridership_2015 = pd.read_csv('csv_originals/2015_platform_usage.csv')
stations = pd.read_csv('stations_cleaned.csv')

In [3]:
# cleaned_2014 = ridership_2014[['Rank', 'Subway/RT Station', 'Totals']].dropna()
stations.head(5)

,Station,Grade,2015 ridership,Ridership
0,COLLEGE,Underground,"47,79",4779
1,DAVISVILLE,Surface,"25,33",2533
2,DUNDAS,Underground,"81,33",8133
3,DUPONT,Underground,"16,14",1614
4,EGLINTON,Underground,"72,75",7275


In [4]:
station_map = {
    'BLOOR (LINE 1, Y-U)': 'BLOOR-YONGE - YU',
    'KENNEDY (LINE 2, B-D)': 'KENNEDY - BD',
    'KENNEDY (LINE 3)': 'KENNEDY - SRT',
    'SHEPPARD-YONGE (LINE 1, Y-U)' : 'SHEPPARD - YU',
    'SHEPPARD-YONGE (LINE 4)' : 'SHEPPARD - SHP',
    'SPADINA (LINE 1, Y-U)' : 'SPADINA - YU',
    'SPADINA (LINE 2, B-D)' : 'SPADINA - BD',
    'ST. GEORGE (LINE 1, Y-U)' : 'ST. GEORGE - YU',
    'ST. GEORGE (LINE 2, B-D)' : 'ST. GEORGE - BD',
    'YONGE (LINE 2, B-D)' : 'BLOOR-YONGE - BD',
    'MAIN STREET' : 'MAIN',
    'MUSEUM' : 'MUSEUM'
}

def sync_station_names(station):
    return station_map.get(station, station)
        
    

In [5]:
def clean_dataset(original):
    cleaned = original[['Rank', 'Subway/RT Station', 'Totals']].dropna()
    
    # Make the stations name a capital
    cleaned['Station'] = cleaned['Subway/RT Station'].str.upper()
    
    # Manually sync up some stations
    cleaned['Station'] = cleaned['Station'].map(sync_station_names)
    
    cleaned['Totals'] = cleaned['Totals'].apply(lambda x: pd.to_numeric(x.replace(',',''), errors='coerce'))
    
    
    return cleaned

In [6]:
cleaned_2014 = clean_dataset(ridership_2014)
cleaned_2014['2014 Total'] = cleaned_2014['Totals']

In [7]:
cleaned_2015 = clean_dataset(ridership_2015)
cleaned_2015['2015 Total'] = cleaned_2014['Totals']

In [10]:
merged = stations.merge(cleaned_2014, on ='Station')
merged = merged.drop(['Subway/RT Station', 'Totals'], 1)
merged = merged.merge(cleaned_2015, on ='Station')
merged = merged.drop(['Subway/RT Station', 'Rank_x', 'Rank_y', 'Totals', '2015 ridership', 'Ridership'], 1)
merged.to_csv('stations_ridership.csv', index=False)